In [1]:
import requests
import re
import pandas as pd
import time
import lxml.html
import openpyxl

from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm import tqdm

In [2]:
custom_header = {
    'referer' : 'http://www.riss.kr/index.do',
    #'referer' : 'http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&exQuery=&exQueryText=&order=&onHanja=&strSort=&p_year1=&p_year2=&iStartCount=0&orderBy=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=bib_t&colName=bib_t&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&query=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5',
    'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'  }

# 검색키워드: 인공지능

In [3]:
#http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&exQuery=regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88pyear%3A2020%E2%97%88pyear%3A2019%E2%97%88pyear%3A2018%E2%97%88pyear%3A2017%E2%97%88pyear%3A2016%E2%97%88pyear%3A2015%E2%97%88pyear%3A2014%E2%97%88pyear%3A2013%E2%97%88pyear%3A2012%E2%97%88pyear%3A2011%E2%97%88pyear%3A2010%E2%97%88language%3Akor%E2%97%88&exQueryText=%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2020%5D%40%40pyear%3A2020%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2019%5D%40%40pyear%3A2019%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2018%5D%40%40pyear%3A2018%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2017%5D%40%40pyear%3A2017%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2016%5D%40%40pyear%3A2016%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2015%5D%40%40pyear%3A2015%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2014%5D%40%40pyear%3A2014%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2013%5D%40%40pyear%3A2013%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2012%5D%40%40pyear%3A2012%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2011%5D%40%40pyear%3A2011%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2010%5D%40%40pyear%3A2010%E2%97%88%EC%9E%91%EC%84%B1%EC%96%B8%EC%96%B4+%5B%ED%95%9C%EA%B5%AD%EC%96%B4%5D%40%40language%3Akor%E2%97%88&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount=0&orderBy=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&query=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5

In [4]:
paper_link_list = []
#285
for i in range(285):
    before_url = "http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&exQuery=regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88pyear%3A2020%E2%97%88pyear%3A2019%E2%97%88pyear%3A2018%E2%97%88pyear%3A2017%E2%97%88pyear%3A2016%E2%97%88pyear%3A2015%E2%97%88pyear%3A2014%E2%97%88pyear%3A2013%E2%97%88pyear%3A2012%E2%97%88pyear%3A2011%E2%97%88pyear%3A2010%E2%97%88language%3Akor%E2%97%88&exQueryText=%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2020%5D%40%40pyear%3A2020%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2019%5D%40%40pyear%3A2019%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2018%5D%40%40pyear%3A2018%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2017%5D%40%40pyear%3A2017%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2016%5D%40%40pyear%3A2016%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2015%5D%40%40pyear%3A2015%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2014%5D%40%40pyear%3A2014%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2013%5D%40%40pyear%3A2013%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2012%5D%40%40pyear%3A2012%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2011%5D%40%40pyear%3A2011%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2010%5D%40%40pyear%3A2010%E2%97%88%EC%9E%91%EC%84%B1%EC%96%B8%EC%96%B4+%5B%ED%95%9C%EA%B5%AD%EC%96%B4%5D%40%40language%3Akor%E2%97%88&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount="
    page = i*10
    after_url = "&orderBy=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&query=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5"
    
    url = before_url + str(page) + after_url
    
    page_req = requests.get(url)
    page_html = page_req.text
    page_soup = BeautifulSoup(page_html, 'html.parser')

    #links.append(page_soup.select('#divContent > div.rightContent > div > div.srchResultW > div.srchResultListW > ul > li > div.cont > p.title > a'))
    links = page_soup.select('#divContent > div.rightContent > div > div.srchResultW > div.srchResultListW > ul > li > div.cont > p.title > a')
    
    for link in links:
        ## Tag안의 텍스트
        print(link.text)
        paper_link_list.append(link.get('href'))
    print(len(paper_link_list))

인공지능 기술분석을 위한 베이지안 추론
텍스트 마이닝 기법을 활용한 인공지능 기술개발 동향 분석 연구: 깃허브 상의 오픈 소스 소프트웨어 프로젝트를 대상으로
딥러닝의 변수 중요도를 이용한 인공지능 기술 분석
소셜 미디어 상에서의 인공지능 관련 사회적 여론에 대한다 범주 감성 분석
심볼릭 인공지능을 위한 R 심볼릭 데이터분석
인공지능 기술 기반 인슈어테크와 디지털보험플랫폼 성공사례 분석: 중국 평안보험그룹을 중심으로
국내외 인공지능형 로봇 개발 및 시장 현황 연구
지능형정보화 시대의 테러유형과 대응방안: 인공지능에 기반한 테러중점
인공지능 로봇의 범죄주체성과 형사책임의 귀속
인공지능 발달에 따른 윤리학의 필요성 연구
10
인공지능 스피커에 대한 소비자 인식과 수용의도: 비수용자를 중심으로
인공지능 기반 자동행정과 법치주의
우리는 왜 인공지능에 대한 통제를 고민해야 하는가?
인공지능(AI) 기반 지능정보사회 시대의 노동시장 변화: 경제사회학적 접근을 중심으로
인공지능 창작물에 대한 법적취급 차별화 방안 검토 : ‘방식주의'의 도입을 중심으로
지능형정보화 시대의 테러유형과 대응방안: 인공지능에 기반한 테러중점
인공지능은 사회적 존재인가?
인공지능 알고리즘의 편향성, 불투명성이 법적 의사결정에 미치는 영향 및 규율 방안
인공지능에 관한 비판적 스케치
인공지능교육에 관한 초중등교사의 인식에 관한 연구
20
인공지능 창작물의 저작권 보호
SW교육 학습자 중심 평가 사례 분석을 통한 인공지능교육의 평가 방향 고찰
인공지능에 의한 판사의 대체 가능성 고찰
인공지능의 안전성과 인간윤리에 대한 법정책적 고찰
인공지능(AI) 교육 플랫폼을 활용한 SW교육 수업안 개발 : 초등학교 고학년을 중심으로
드라마 〈퍼슨 오브 인터레스트〉 속 인공지능의 의미 연구
인공지능 로봇의 형사책임 ―논의방향의 설정에 관한 몇 가지 발전적 제언―
인공지능과 계약법
인공지능에 대한 전자인 제도 도입의 필요성과 실현방안에 관한 고찰
호주의 관점에서 바라본 인공지능, 윤리 및 규제 과제
30
인공지능을 이

인공지능의 위험의 특성과 법적 규제방안
시각예술에서 인공지능과 빅데이터의 역할
인공지능에 의한 계약의 효력
인공지능을 이용하여 체결한 소비자계약과 청약철회에 관한 연구
인공지능에 의한 소비자권익 침해에 관한 유형과 법적 과제
인공지능과 불법행위책임 법리
인공지능과 법인격
인공지능에 대한 민사책임 법리
인공지능(AI)기술을 활용한 저작물의 법적 문제에 관한 연구
인공지능 창작물에 대한 법적취급 차별화 방안 검토
250
인공지능 로봇의 특허침해에 따른 책임
인공지능과 예배의 초월적 경험으로 자기희생의 윤리적 가능성 연구
인공지능에 의한 차별과 그 책임 논의를 위한 예비적 고찰
“사법시스템과 사법환경에서의 인공지능 이용에 관한 유럽 윤리헌장”의 검토 - 민사사법절차에서의 인공지능 도입 논의와 관련하여 -
인공지능 창작물에 대한 저작권의 주체
융복합 시대에 일부 보건계열 전공 학생들의 의료용 인공지능에 대한 기대도
인공지능의 역사
인공지능 규제 정책에 관한 연구:  주요국의 규제 정책 사례를 중심으로
세법상 법률 인공지능의 한계와 그 영향에 관한 試論的 考察
대학원 인공지능교육의 방향탐색: IPA를 활용하여
260
프래그머티즘과 인공지능 시대의 예술 경험 - 인공지능 시대, 예술의 종말과 미적 경험의 회복
중국 인공지능의 국가적 수용과 그 의미: 2015-2017 국가 공문 분석을 중심으로
특허법상 인공지능 창작물의 발명 해당 여부
머신러닝 인공지능의 법 분야 적용의 현재와 미래
4차 산업혁명에 대응한 지적재산권 제도의 활용 - ­‘인공지능 창작물 보호제도'를 중심으로­ -
의료 인공지능에 대한 형법적 고찰-왓슨(Watson)을 중심으로-
머신러닝 인공지능과 인간전문직의 협업의 의미와 법적 쟁점
인문콘텐츠의 확장을 위한 인공지능인문학 시론(試論)
차별에서 공정성으로:인공지능의 차별 완화와 공정성 제고를 위한 제도적 방안
“사법시스템과 사법환경에서의 인공지능 이용에 관한 유럽 윤리헌장”의 검토 - 민사사법절차에서의 인공지능 도입 논의와 관련하여 -
270
예술에서 살펴

인공지능 기반의 TensorFlow 그래픽 사용자 인터페이스 개발에 관한 연구
인공지능을 이용한 급성 뇌졸중 환자의 재원일수 예측모형 개발
인공지능 분야 국방 미래기술에 관한 실증연구
인공지능인문학: If의 미래학에서 As-If의 철학으로
‘인공지능', ‘기계학습', ‘딥 러닝' 분야의 국내 논문 동향 분석
인공지능에 대한 사용자 관점의 이해와 이를 응용한 이미지 효과 및 영상 편집 프로그램에 대한 고찰
인공지능 기계학습 방법 비교와 학습을 통한 디지털 신호변화
초등학생의 인공지능 교육을 위한 교수 학습 모델 개발 및 적용
인공지능 ‘기술-윤리원칙-산업-제도'의 매트릭스와 적용
인공지능 위험정책 필요성에 대한 실증적 고찰: AHP 조사결과를 중심으로
490
인공지능과 인간의 마음
인공지능과 디지털 기술 발달에 따른 트랜스/포스트휴머니즘에 관한 학제적 연구
인공지능의 시대, 프로페셔널리즘의 위기와 교양교육의 방향
인(人) 이외의 존재에 대한 법인격 인정과 인공지능의 법적 지위에 관한 소고
인공지능 윤리규범과 규제 거버넌스의 현황과 과제
인공지능 기반 자동화행정의 주요 쟁점에 관한 연구
인공지능의 몸: 현상학적 고찰
영화 속 인공지능의 현상학적 인간성 -영화 "트랜센던스"와 "그녀"를 중심으로-
로봇 활용 인공지능 교육 프로그램 개발과 적용에 관한 연구
인공지능 기반의 생성적 로고디자인 사례 연구   - 전통적 방식의 로고디자인 제작 사례와 비교하여 -
500
인공지능에 의한 저작물 이용 및 창작에 대한 법적 검토와 시사점
인공지능 기반 수요예측 기법의 리뷰
인공지능 기술의 행정 활용에 관한 연구동향 및 쟁점 분석
인공지능과 인간의 마음
인간과 인공지능 로봇 캐릭터의 비교 연구-너도 인간이니? 기반으로
인공지능의 개인정보 자동화 처리가 야기하는 차별 문제에 관한 연구
인공지능 시대를 ‘살아내기' 위한 미래 마인드와 책임적 리더십에 대한 고찰
인공지능과 블록체인의 도입이 사이버 안보의 공 · 수 비대칭 구도에 갖는 의미
인공지능과 환경윤리 - 인공지능의 도덕적, 법

인공지능을 적용한 소형 모바일 로봇을 위한 물체 및 환경 인식 연구
광고 및 미디어 산업 분야의 인공지능(AI) 활용 전략
초등 교사들의 인공지능에 관한 교육적 인식
인공지능 기반 자동화행정의 주요 쟁점에 관한 연구
인공지능 기반 질소산화물 배출량 예측을 위한 연구모형 개발
인공지능 시대의 가족과 불교의 수행공동체
인공지능을 생명이라 할 수 있는가?
인공지능과 개인정보보호 - 불필요한 갈등 증가에 대한 관견(管見) -
인공지능을 이용한 대장폴립의 진단: 미소폴립의 정확한 분류
인공지능 시대 비판적 반성 사고를 위한 P4C와 미디어 리터러시 융합 통일교육 방안
710
바르트의 『신화론』을 통한 인공지능 담론 연구
인공지능을 적용한 시력데이터 분석 : 기계학습 모델간 비교 및 최적모델제시
인공지능(AI) 스피커와의 상호작용이 소비자 심리에 미치는 영향
AI(인공지능)에 대한 신학적 담론의 형성 및 방향 모색
인공지능시대에 의사의 역할
인공지능(Artificial Intelligence: AI) 스피커 이용자의 인지지도와 잠재된 프라이버시 인식
인공지능 기법을 활용한 법률안 예측 모델 연구
인공지능과 내과 의사의 미래
인공지능 시대 “Historia, Quo Vadis?”
인공지능 시대의 저작물 대량 디지털화에 관한 소고
720
인공지능을 활용한 다중 생체신호 분석 기반 스마트 감정 관리 시스템
인공지능이 따라하지 못할 인문학적 뇌와 인간의 양면성(창조성과 한계)에 도전하는 인공지능
인공지능 기법으로 스마트 플러그를 이용한 제품 자동분류에 관한 연구
인공지능을 소재로 한 영화의 갈등의 특징
인공지능시대와 동아시아의 관계론 사유 : 책임소재와 책임배분의 문제를 중심으로
공감적 인공지능상담가 활용에 대한 미래 전망
인공지능과 예술창작 활동의 융복합 사례분석 및 특성 연구
인공지능과 의사의 미래
인공지능의 지식재산권 보호에 관한 연구
인공지능의 문장 부호번역 패턴에 따른 번역 전략연구
730
인공지능을 활용한 영어 학습용 챗봇 시스템 개발 방안 연구
인공지능망과 뉴로퍼지

효과적인 인공지능 교육을 위한 컴퓨터 게임의 활용
AI(인공지능)과 IOT(사물인터넷)을 통한 Smart health Care Service 산업 활성화 방안
7포커 인공지능 시뮬레이터 구현
폭염기간 인공지능 센서를 이용한 아파트 정전예방 방안
생각한다: 인공지능(AI) 시대의 음악 연구 단상
채용 전형에서 인공지능 기술 도입이 입사 지원의도에 미치는 영향
미국과 중국의 인공지능 특허 동향 비교분석
웨어러블 동작센서와 인공지능 학습모델 기반에서 행동인지의 개선
자율 시스템, 인공지능과 로봇: 형법의 관점에서 지향성은?
미래는 초인류와 인공지능이 지배하는 세상인가
940
국내 하수처리시설에 인공지능기술 적용을 위한 사례 연구
특허정보를 이용한 인공지능 기술 동향 분석
학술기사 : 인공지능 계측관리시스템을 이용한 수상구조물 진수
연하장애 분야의 인공지능 적용
특허분석을 통한 인공지능 기술 분야 경쟁력 분석:특허 시장성과 기술력 질적 분석을 중심으로
초등 생물분류 학습에서 인공지능 융합교육의 적용 사례 연구
텍스트 마이닝 기법을 활용한 인공지능과 헬스케어 융․복합 분야 연구동향 분석
기어박스의 예측정비를 위한 인공지능 기반의 이상탐지 및 결함분류에 관한 연구
영어발음 향상을 위한 실시간 인공지능 입모양 인식 프로그램 개발
보안관제 정탐률 향상을 위한 인공지능 알고리즘 연구
950
키워드 분석을 통한 인공지능 분야의 유망 기술 연구
디자인씽킹 프로세스 기반의 인공지능(AI) 교육 프로그램 적용 효과분석
다요소 가중 평균법을 이용한 인공지능 기술 개발전략 연구
초고압 전력기기의 부분방전 인공지능 인식 알고리즘 적용 연구
PC 사용자를 위한 인공지능 기반 거북목 경고 시스템 개발
상부위장관 암 진단을 위한 인공지능 분석, 어디까지 발전하였는가?
초등영어 예비교사들이 개발한 인공지능 챗봇
4차 산업혁명에 따른 쌍방향 인공지능을 통한 영어학습의 변화
기계학습 알고리즘 기반의 인공지능 장기 게임 개발
성공적 갈등관리를 위한 인공지능 활용 전략 연구
960
온라인-오프라인의 연

...

일본의 한적 정보화와 코퍼스 구축 현황
로봇수술로 인한 의료과오 민사책임에 있어 과실 판단의 문제
4차 산업혁명시대 진로교육의 방향 모색
지방자치단체의 자주재원확보방안에 관한 연구
특허 IPC Network 분석 및 Word2Vec 기반 물류 자동화 기술동향에 관한 연구
미학적 주체로서의 포스트휴먼 ?
한국소설의 비인간 전환과 탈인간중심주의
기본소득제도의 논의배경과 한계에 관한 연구
고전서사 연구에서 연결성에 대한 논의의 현단계― 고전소설의 디지털 인문학적 연구 사례 검토를 중심으로 ―
효과적인 CUDA 디버깅을 위한 CUDA 커널 분리 소프트웨어 도구
1870
합성곱 신경망을 이용한 이미지 기반 화재 감지 시스템의 구현
융합교양교육이 공학인재의 소프트스킬 향상에 미치는 효과
신라 신왕의 장생불사론 : 신기(神器), 수(壽), 그리고 왕위계승법칙
듀이 미학 관점에서 창의적 소프트웨어 교육에 대한 소고 : 아트코딩
역전파 알고리즘을 이용한 대한민국 국민연금 고갈 시기 예측 연구
iClass에 기반한 [인성] 교양강좌 콘텐츠 구성과 효과
디자인 창의성 평가를 위한 테스트 모델 개발과 활용방안 연구
교사의 앱 개발 교육에 대한 관심도 분석 : 관심중심수용모형(CBAM)을 중심으로
CNN을 활용한 영상 기반의 화재 감지
증강현실기반 O2O서비스용 지역문화산업 정보포털 App 구축에 관한 연구
1880
베이지안 신경망을 이용한 순차적 다중작업 학습 최적화
포스트 휴먼에 대한 영화적 성찰 -2000년 이후 스티븐 스필버그(Steven Spielberg)의 SF 영화를 중심으로
객체지향 데이터베이스의 다중계승을 위한 동시성 제어 기법 개발
XOB : XMDR 기반의 온톨로지 생성 시스템
구조방정식 분석을 통한 음성비서 서비스 수용에 대한 영향 요인 연구
한국어 자동 띄어쓰기 시스템을 위한 학습 데이터의 효과적인 표현 방법
정보기술의 발전이 종교생활에 미치는 영향
문학과 K-팝, 유투브 르네상스
산업연관분석을 이용한 정보보호 산업의 경제 파급효과 분석
인천공항 무인로봇의 체

군사용 드론의 수용의도에 영향을 미치는 요인에 관한 연구
사물인터넷의 사회적 영향 : 시나리오 플래닝을 통한 일자리 영향 전망
딥 러닝을 이용한 포트폴리오 구성 전략
고빈도 거래와 관련한 미국의 최근 규제동향 및 시사점
LiAS: 점진적 사전 확장과 기계학습을 활용한 선형구조의 언어정보 부착 시스템
공공기관 통번역 팀 운영에 관한 통시적 고찰
의미 기반의 지식모델 통합과 탐색에 관한 연구
디지털 교역과 통상규범
ID3 알고리즘 기반의 귀납적 추론을 활용한 인터넷 기업 비즈니스 모델의 성공과 실패에 영향을 미치는 요인에 관한 연구
SNS 사용자 특성과 확증 편향을 통한 자동화된 팩트체킹의 가능성 : 정치인 관련 트윗 데이터를 중심으로
2090
NIDS의 비정상 행위 탐지를 위한 단일 클래스 분류성능 평가
워드 임베딩을 이용한 아마존 패션 상품 리뷰의 사용자 감성 분석
메이커 교육에 디자인 사고 적용 연구
사물인터넷(IoT) 헬스케어 서비스 법제도 개선 방향
스마트 시티 인프라 관리를 위한 사물인터넷 식별체계 및 엣지 게이트웨이 아키텍처
기계에서 장치로 ― 헤겔과 플루서의 “기계철학” ―
인지의미론을 활용한 가치-범주 기억 시스템 연구
희생양극법을 적용한 철도 레일의 방식효과 모니터링 연구
데이터 분석적 사고력 향상을 위한 딥러닝 기반 학습 시스템 개발 연구
이종 공작기계 운용 관리를 위한 분산 스마트 데이터 모니터링 시스템 개발
2100
IoT 정보 수집을 위한 확률 기반의 딥러닝 클러스터링 모델
M&S와 AI간의 유기적 통합을 위한 시간기반 전문가 시스템 설계
학교 변화와 독서 교육
RNN을 이용한 제2형 당뇨병 예측모델 개발
딥러닝 모델을 이용한 교육사회통념 가설검증 예측 시스템의 제안
핀테크 관련 금융사고의 법적책임 연구
블록체인 노드 신뢰성 향상을 위한 사용자 검증 시스템
블록체인 기술의 법적 쟁점과 운용방안에 관한 연구
CNN 알고리즘의 개선을 통한 대형 이미지에서의 객체식별
마커기반 모션캡쳐의 환경극복에 관한 연구
2110
정부 3.0 시대의 다기관 

개인 맞춤형 관광 서비스 플랫폼 모델에 관한 연구
VR을 도입한 생태학적 패러다임 건축 공간 연구
사운드 설치 작품의 의미 축조와 현상학적 이해 - <까마귀의 살해>를 중심으로
칸트철학에서 지식과 언어
4차 산업혁명과 새로운 산업발전 공간정책 방향: 플랫폼 전략의 적용 검토*
4차 산업혁명 시대의 교육적 함의와 상상력을 통한 신앙양육의 패러다임
멀티코어 CPU 환경하에서 능률적인 네트워크 관리를 위한 유전알고리즘을 이용한 국부적 RED 조정 기법
건물 내에서 화재시의 대피 시뮬레이션 설계 및 구현
4차 산업혁명기 협력적 규제 거버넌스의 필요성과 전문가의 역할
협업필터링시스템의 발전 및 활용 방안 연구
2320
전자적의사표시에 대한 민법상 적용문제에 관한 연구
Double DQN 기반의 끊김 없는 적응형 비디오 스트리밍 기법
웹3.0 아이덴티티 디자인에 관한 연구
감정형용사를 통한 자연풍경의 표상적 호감구조에 관한 연구
다중 매트릭스 분석 기법을 이용한 최적 건축공법 선정의사결정지원 모델
다중 매트릭스 분석 기법을 이용한 최적 건축공법 선정 의사결정지원 모델
U-Learning을 위한 위치 기반 서비스로서의 상황 인식 기술
儒家的 삶의 脫現代的 含意
사이버 심리전의 프로퍼갠더 전술과 권위주의 레짐의 샤프파워 : 러시아의 심리전과 서구 민주주의의 대응
AHP분석을 활용한 향후 디지털 마케팅 구성요인의 중요도 연구: 부산국제광고제 애드텍 2017 사례를 중심으로
2330
中韓(韓中) 번역을 통해 본 구글번역(GNMT)의 문제
인간 단백질 분석을 위한 빅 데이타 기반 RMF 방법
인슈어테크 사례를 통한 비즈니스모델에 대한 연구
영화산업에서 빅데이터의 활용방안 연구 - ‘넷플릭스(Netflix)' 분석기술 중심으로
디지털 인문학의 관점에서 본 디지털 불교학
이원분류기법을 이용한 소규모 교량 상부형식선정 모형에 관한 연구
다색(多色)의 발광(發光)효과가 있는 광섬유직물 카시트 연구
군집분석과 유전자 알고리즘을 활용한 투자자 거래정보 기반 포트폴리오 투자전략
스마트 팩토리

비트코인(Bitcoin) 악용 범죄의 대응방안에 관한 연구
공공데이터 개방표준 데이터의 품질평가
통일대비 남북 접경지역 발전 방안에 관한 연구
AI융합 교육을 위한 교사 교육과정 개발
디자인 언어개념 정립을 통한 인더스트리얼디자인 언어 교육콘텐츠 연구
주제어 네트워크 분석을 활용한 비서 연구 동향
가시광선-근적외선 반사스펙트럼을 이용한 쇠고기의 신선도 평가
AI의 예측능력과 재범예측알고리즘의 헌법 문제- State v. Loomis 판결을 중심으로
디지털 기술의 융합을 통한 패션 디자인 개발에 관한 연구 -빅데이터와 3D 가상착의 프로그램을 중심으로-
찰스 바비지의 계산기관에서 인간, 기계, 생각의 관계성
2550
CNN Mobile Net 기반 악성코드 탐지 모델에서의 학습 데이터 크기와 검출 정확도의 상관관계 분석
제4차 산업혁명과 출판의 미래 모델 연구: 기술 혁신 환경의 출판 모델과 적응 전략을 중심으로
생태공동체 모델 구축을 위한 인간, 자연, 기술 개념 연구
블록체인 기술의 활용 현황과 발전 방안에 관한 연구: 해외의 암호화폐 정책을 중심으로
국내 디지털 사이니지 광고PR 산업의 성장 전략 연구 광고PR 전문가 대상의 FGI, MDS 및 IPA 매트릭스 분석을 중심으로
공생의 기호학 – 찰스 퍼어스(C.S. Peirce)와 야콥 폰 윅스퀼(Jacob von Uexküll) 그리고 앤디 클라크(Andy Clark)를 통해 구성하는 공생의 존재론
완전 자율주행차에 적용된 인터랙티브 이모티콘이 자동차 디자인 인식에 미치는 영향
AHP를 활용한 4차 산업혁명 대응을 위한 우리나라 기업의 기술 적용 우선순위 및 가중치에 대한 연구
들리는 음악에서 들리지 않는 음악으로: 21세기 디지털 테크놀로지와 개념음악에 관한 소고
증류탑을 위한 머신러닝 기반 플랫폼 개발
2560
중소제조공장을 위한 제조데이터 분석 플랫폼의 모바일 HW 유저 인터페이스 연구 -현장 친화형 웨어러블 골전도 무선이어폰 디바이스의 개발을 중심으로-
한국 중학생의 컴퓨팅 사고력 분석
4차 산업 

빅데이터 시대 생성정보의 처리 체계 - 추론된 정보의 처리에 관한 우리 개인정보보호법의 규율과 개선 방안에 관한 고찰 -
제4차 산업혁명 시대의 근로조건 규율–스마트워크 근로자를 중심으로–
디지털시대의 빅데이터와 경쟁법
과학기술 및 정보화기술 시대의 스포츠 세계에서 휴머니즘에 대한 전망
4차 산업혁명시대, 공연예술의 융⋅복합 사례연구
4차 산업혁명시대 ‘문화원형 소재 오픈소스화를 위한디지털콘텐츠화사업'의 필요성
유정(有情)과 무정(無情)의 간극에 대해‒ 감각의 확장에 대한 소고(小考) ‒
儒家的 삶의 脫現代的 含意 -敬堂 張興孝의 工夫論과 持敬의 삶을 중심으로-
4차 산업혁명과 기계 의료 윤리
미래 사회 대비 국어 문식성 교육의 역할과 과제
2770
빌렘 플루서의 예술론
딥러닝 기반의 다층 퍼셉트론을 이용한 심장병 예측 연구
홀로렌즈를 이용한 증강현실 육성 시뮬레이션 개발
제조업 과학기술 융합구조 분석 및 국가 표준정책 연구
자율주행자동차 교통사고의 형사책임
대학 글쓰기 교육에서 자기 서사 정립의 중요성과 ‘쓰기' 개념 재고의 필요성 - 자기 소개 영상 제작 수업 실례를 중심으로
작업 준비비용 최소화를 고려한 강화학습 기반의 실시간 일정계획 수립기법
연기의 근본―몸의 자연성
4차산업혁명과 일터의 민주화한국과 독일: 두 나라 이야기
캐릭터 움직임 분석 도구로서의 애니메이션 원리 연구 : 라반 움직임 분석(LMA)과의 비교를 중심으로
2780
현대 패션쇼에 나타난 포스트휴먼의 내적 특성 연구
스마트 공유경제사회에서 빅데이터의 법률관계에 관한 小考
4차 산업혁명과 불교미술의 새로운 방향성 모색
휴머노이드 로봇의 언캐니(Uncanny) 이미지 연구
애니메이션 플랫폼의 기술적 진화와 정책적 관점에 대한 연구
해외 글로벌 패션기업의 디지털전환 전략 동향 분석
NFC 기반 FIDO(Fast IDentity Online) 및 2 Factor 기술과 허가형 분산원장 블록체인을 이용한 모바일 갤러리 경매 방안 제안
도예 활동 특성이 직업 활동 의도에 미치는 영향 연구 - 일본 

In [5]:
paper_link_list

['/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=d660bc7a575db880d18150b21a227875',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=7ecb0602bc54ba777ecd42904f0c5d65',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=2018b7600d122280d18150b21a227875',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=51fa16c99f2e25f5b7998d826d417196',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=14cd6e1a3ee39df14884a65323211ff0',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=b19e182b52eecc5de9810257f7042666',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=4a7c248d04d5da9447de9c1710b0298d',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=fd4c3aaec94e7bedb7998d826d417196',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=fdef80aa3d3c19f1d18150b21a227875',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c

In [6]:
len(paper_link_list)

2848

In [7]:
paper_url = []

for paper_link in tqdm(paper_link_list):
    tmp_url = "http://www.riss.kr" + paper_link
    paper_url.append(tmp_url)

100%|██████████████████████████████████████████████████████████████████████████████████████| 2848/2848 [00:00<?, ?it/s]


In [8]:
paper_url

['http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=d660bc7a575db880d18150b21a227875',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=7ecb0602bc54ba777ecd42904f0c5d65',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=2018b7600d122280d18150b21a227875',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=51fa16c99f2e25f5b7998d826d417196',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=14cd6e1a3ee39df14884a65323211ff0',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=b19e182b52eecc5de9810257f7042666',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=4a7c248d04d5da9447de9c1710b0298d',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=fd4c3aaec94e7bedb7998d826d417196',
 'http://www.ris

In [9]:
len(paper_url)

2848

In [10]:
write_wb = openpyxl.Workbook()
write_ws = write_wb.active

In [11]:
"""
write_ws = write_wb.active
write_ws['A1'] = 'title_kor'
write_ws['B1'] = 'title_eng'
write_ws['C1'] = 'author'
write_ws['D1'] = 'book'
write_ws['E1'] = 'year'
write_ws['F1'] = 'keyword'
write_ws['G1'] = 'abstract_kor'
write_ws['H1'] = 'abstract_eng'
"""

"\nwrite_ws = write_wb.active\nwrite_ws['A1'] = 'title_kor'\nwrite_ws['B1'] = 'title_eng'\nwrite_ws['C1'] = 'author'\nwrite_ws['D1'] = 'book'\nwrite_ws['E1'] = 'year'\nwrite_ws['F1'] = 'keyword'\nwrite_ws['G1'] = 'abstract_kor'\nwrite_ws['H1'] = 'abstract_eng'\n"

In [12]:
write_ws = write_wb.active
write_ws['A1'] = 'title'
write_ws['B1'] = 'author'
write_ws['C1'] = 'book'
write_ws['D1'] = 'year'
write_ws['E1'] = 'keyword'
write_ws['F1'] = 'abstract_kor'
write_ws['G1'] = 'abstract_eng'

In [13]:
start = time.time()

for i, url in enumerate(paper_url):
    res = requests.get(url, headers = custom_header)
    element = lxml.html.fromstring(res.text)

    try:
        tmp_title = element.cssselect('h3.title')
        tmp_title = tmp_title[0].text_content()
        tmp_title = tmp_title.split("=")
        title_kor = tmp_title[0].strip()
        #title_eng = tmp_title[1].strip()

        tmp_author = element.cssselect('div.infoDetailL > ul > li:nth-child(1) > div > p')
        author = tmp_author[0].text_content().strip()

        tmp_book = element.cssselect('div.infoDetailL > ul > li:nth-child(3) > div > p')
        book = tmp_book[0].text_content().strip()

        tmp_year = element.cssselect('div.infoDetailL > ul > li:nth-child(5) > div > p')
        year = tmp_year[0].text_content().strip()

        tmp_keyword = element.cssselect('div.infoDetailL > ul > li:nth-child(7) > div > p')
        keyword = tmp_keyword[0].text_content().strip()

        tmp_abstract = element.cssselect('div.text.off')
        abstract_kor = tmp_abstract[0].text_content().strip()
        abstract_eng = tmp_abstract[1].text_content().strip()

        print(i, 'paper clear')

    except:
        print(i, 'ERROR')
        continue


    write_ws.append([title_kor, author, book, year, keyword, abstract_kor, abstract_eng])
    write_wb.save('D:/Jupyter/museong/3. Crawling/paper_ai.xlsx')
    
print("Running Time : ", time.time() - start)

0 paper clear
1 paper clear
2 paper clear
3 paper clear
4 paper clear
5 paper clear
6 paper clear
7 paper clear
8 paper clear
9 paper clear
10 paper clear
11 paper clear
12 paper clear
13 paper clear
14 paper clear
15 paper clear
16 paper clear
17 paper clear
18 paper clear
19 paper clear
20 paper clear
21 paper clear
22 paper clear
23 paper clear
24 paper clear
25 paper clear
26 paper clear
27 paper clear
28 paper clear
29 paper clear
30 paper clear
31 paper clear
32 paper clear
33 paper clear
34 paper clear
35 paper clear
36 paper clear
37 paper clear
38 paper clear
39 paper clear
40 paper clear
41 paper clear
42 paper clear
43 paper clear
44 paper clear
45 paper clear
46 paper clear
47 paper clear
48 paper clear
49 paper clear
50 paper clear
51 paper clear
52 paper clear
53 paper clear
54 paper clear
55 paper clear
56 paper clear
57 paper clear
58 paper clear
59 paper clear
60 paper clear
61 paper clear
62 paper clear
63 paper clear
64 paper clear
65 paper clear
66 paper clear
67 pa

521 paper clear
522 paper clear
523 paper clear
524 paper clear
525 paper clear
526 paper clear
527 paper clear
528 paper clear
529 paper clear
530 paper clear
531 paper clear
532 paper clear
533 paper clear
534 paper clear
535 paper clear
536 paper clear
537 paper clear
538 paper clear
539 paper clear
540 paper clear
541 paper clear
542 paper clear
543 paper clear
544 paper clear
545 paper clear
546 paper clear
547 paper clear
548 paper clear
549 paper clear
550 paper clear
551 paper clear
552 paper clear
553 paper clear
554 paper clear
555 paper clear
556 paper clear
557 paper clear
558 paper clear
559 paper clear
560 paper clear
561 paper clear
562 paper clear
563 paper clear
564 paper clear
565 paper clear
566 paper clear
567 paper clear
568 paper clear
569 paper clear
570 paper clear
571 paper clear
572 paper clear
573 paper clear
574 paper clear
575 paper clear
576 paper clear
577 paper clear
578 ERROR
579 paper clear
580 paper clear
581 paper clear
582 paper clear
583 paper clea

1053 paper clear
1054 paper clear
1055 paper clear
1056 paper clear
1057 paper clear
1058 paper clear
1059 paper clear
1060 paper clear
1061 paper clear
1062 paper clear
1063 paper clear
1064 paper clear
1065 paper clear
1066 paper clear
1067 paper clear
1068 paper clear
1069 paper clear
1070 paper clear
1071 paper clear
1072 paper clear
1073 paper clear
1074 paper clear
1075 paper clear
1076 paper clear
1077 paper clear
1078 paper clear
1079 paper clear
1080 paper clear
1081 paper clear
1082 paper clear
1083 paper clear
1084 paper clear
1085 paper clear
1086 paper clear
1087 paper clear
1088 paper clear
1089 paper clear
1090 paper clear
1091 paper clear
1092 paper clear
1093 paper clear
1094 paper clear
1095 paper clear
1096 paper clear
1097 paper clear
1098 paper clear
1099 paper clear
1100 paper clear
1101 paper clear
1102 paper clear
1103 paper clear
1104 paper clear
1105 paper clear
1106 paper clear
1107 paper clear
1108 paper clear
1109 paper clear
1110 paper clear
1111 paper cle

1537 paper clear
1538 paper clear
1539 paper clear
1540 paper clear
1541 paper clear
1542 paper clear
1543 paper clear
1544 paper clear
1545 paper clear
1546 paper clear
1547 paper clear
1548 ERROR
1549 paper clear
1550 paper clear
1551 paper clear
1552 paper clear
1553 paper clear
1554 paper clear
1555 paper clear
1556 paper clear
1557 paper clear
1558 paper clear
1559 paper clear
1560 paper clear
1561 paper clear
1562 paper clear
1563 paper clear
1564 paper clear
1565 paper clear
1566 paper clear
1567 paper clear
1568 paper clear
1569 paper clear
1570 paper clear
1571 paper clear
1572 paper clear
1573 paper clear
1574 paper clear
1575 paper clear
1576 paper clear
1577 paper clear
1578 paper clear
1579 paper clear
1580 paper clear
1581 paper clear
1582 paper clear
1583 paper clear
1584 paper clear
1585 paper clear
1586 paper clear
1587 paper clear
1588 paper clear
1589 paper clear
1590 paper clear
1591 paper clear
1592 paper clear
1593 paper clear
1594 paper clear
1595 paper clear
159

2023 paper clear
2024 paper clear
2025 paper clear
2026 paper clear
2027 paper clear
2028 ERROR
2029 paper clear
2030 paper clear
2031 paper clear
2032 ERROR
2033 paper clear
2034 paper clear
2035 paper clear
2036 paper clear
2037 paper clear
2038 paper clear
2039 paper clear
2040 paper clear
2041 paper clear
2042 paper clear
2043 paper clear
2044 paper clear
2045 paper clear
2046 paper clear
2047 paper clear
2048 paper clear
2049 paper clear
2050 paper clear
2051 ERROR
2052 paper clear
2053 paper clear
2054 paper clear
2055 paper clear
2056 paper clear
2057 paper clear
2058 paper clear
2059 paper clear
2060 paper clear
2061 paper clear
2062 paper clear
2063 paper clear
2064 paper clear
2065 paper clear
2066 paper clear
2067 paper clear
2068 paper clear
2069 paper clear
2070 ERROR
2071 paper clear
2072 paper clear
2073 paper clear
2074 paper clear
2075 paper clear
2076 paper clear
2077 paper clear
2078 paper clear
2079 paper clear
2080 paper clear
2081 paper clear
2082 paper clear
2083

2510 paper clear
2511 paper clear
2512 paper clear
2513 paper clear
2514 paper clear
2515 paper clear
2516 paper clear
2517 paper clear
2518 paper clear
2519 paper clear
2520 paper clear
2521 paper clear
2522 paper clear
2523 paper clear
2524 paper clear
2525 paper clear
2526 paper clear
2527 paper clear
2528 paper clear
2529 paper clear
2530 paper clear
2531 paper clear
2532 paper clear
2533 paper clear
2534 paper clear
2535 paper clear
2536 paper clear
2537 paper clear
2538 paper clear
2539 paper clear
2540 paper clear
2541 paper clear
2542 paper clear
2543 paper clear
2544 paper clear
2545 paper clear
2546 paper clear
2547 paper clear
2548 paper clear
2549 paper clear
2550 paper clear
2551 paper clear
2552 paper clear
2553 paper clear
2554 paper clear
2555 paper clear
2556 paper clear
2557 paper clear
2558 paper clear
2559 paper clear
2560 paper clear
2561 paper clear
2562 paper clear
2563 paper clear
2564 paper clear
2565 paper clear
2566 paper clear
2567 paper clear
2568 paper cle